## Simple Parallel Workflow

In [ ]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from typing_extensions import Annotated
from operator import add



In [ ]:
class BatsmanState(TypedDict):
  runs: int
  balls: int
  fours: int
  sixes: int
  
  sr: float
  bpb: float
  boundary_percentage: float
  summary: str

In [ ]:
# ==========================
# Calculate Strike Rate Node
# ==========================
def calculate_sr(state: BatsmanState) -> BatsmanState:
  sr = (state["runs"] / state["balls"]) * 100
  return {"sr": sr}
  # return state


# =================================
# Calculate Balls per Boundary Node
# =================================
def calculate_bpb(state: BatsmanState) -> BatsmanState:
  bpb = (state["fours"] + state["sixes"]) / state["balls"]
  return {"bpb": bpb}
  # return state


# =================================
# Calculate Boundary Percentage Node
# =================================
def calculate_boundary_percentage(state: BatsmanState) -> BatsmanState:
  # Boundary runs ka percentage in total runs
  boundary_runs = (state["fours"] * 4) + (state["sixes"] * 6)
  boundary_percentage = (boundary_runs / state["runs"]) * 100
  return {"boundary_percentage": boundary_percentage}
  # return state

# =================================
# Show Summary Node
# =================================
def summary(state: BatsmanState) -> BatsmanState:
  summary_text = f"Runs: {state['runs']}, Balls: {state['balls']}, Fours: {state['fours']}, Sixes: {state['sixes']}, SR: {state['sr']:.2f}, BPB: {state['bpb']:.2f}, Boundary %: {state['boundary_percentage']:.2f}%"
  return {"summary": summary_text}
  # return state



In [ ]:
# graph = StateGraph[BatsmanState, None, BatsmanState, BatsmanState](BatsmanState)

graph = StateGraph(BatsmanState)

# Add nodes
graph.add_node("calculate_sr", calculate_sr)
graph.add_node("calculate_bpb", calculate_bpb)
graph.add_node("calculate_boundary_percentage", calculate_boundary_percentage)
graph.add_node("summary", summary)

# Add edges
graph.add_edge(START, "calculate_sr")
graph.add_edge(START, "calculate_bpb")
graph.add_edge(START, "calculate_boundary_percentage")

graph.add_edge("calculate_sr", "summary")
graph.add_edge("calculate_bpb", "summary")
graph.add_edge("calculate_boundary_percentage", "summary")
graph.add_edge("summary", END)

# Compile the graph
workflow = graph.compile()



In [ ]:
# Visualize the graph
from IPython.display import Image, display

try:
    display(Image(workflow.get_graph().draw_mermaid_png()))
except Exception:
    # Agar mermaid visualization kaam nahi kare toh ASCII representation dikhayen
    print(workflow.get_graph().draw_ascii())


In [ ]:
# Alternative: Text representation
print("Graph Structure:")
print("=" * 50)
print(workflow.get_graph())


In [ ]:
initial_state = {"runs": 100, "balls": 100, "fours": 10, "sixes": 5}

# Run the workflow
final_state = workflow.invoke(initial_state)
print(final_state)



In [ ]:
# Manual Verification
print("\n" + "="*60)
print("MANUAL VERIFICATION:")
print("="*60)

runs, balls, fours, sixes = 100, 100, 10, 5

# Strike Rate
sr_manual = (runs / balls) * 100
print(f"Strike Rate: {sr_manual:.2f} (Expected: 100.00)")

# Balls per Boundary
bpb_manual = (fours + sixes) / balls
print(f"Balls per Boundary: {bpb_manual:.2f} (Expected: 0.15)")

# Boundary Percentage
boundary_runs = (fours * 4) + (sixes * 6)
boundary_pct_manual = (boundary_runs / runs) * 100
print(f"Boundary %: {boundary_pct_manual:.2f}% (Expected: 70.00%)")
print(f"  → Boundary runs: {boundary_runs} out of {runs} total runs")

print("\n✅ Ab calculations accurate hain!")
